In [12]:
!pip install gradio==3.43.0
!pip install tensorflow pymorphy3 nltk joblib

In [13]:
import nltk
nltk.download('stopwords')

import joblib
from tensorflow.keras.models import load_model
import pymorphy3
import re
from nltk.corpus import stopwords
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
MODEL_PATH = "/content/sentiment_model.h5"
VECTORIZER_PATH = "/content/tfidf_vectorizer.joblib"
ENCODER_PATH = "/content/label_encoder.joblib"

model = load_model(MODEL_PATH)
vectorizer = joblib.load(VECTORIZER_PATH)
label_encoder = joblib.load(ENCODER_PATH)

morph = pymorphy3.MorphAnalyzer()
russian_stopwords = stopwords.words("russian")

In [15]:
def preprocessing_text(str_row):
    if not isinstance(str_row, str):
        str_row = ""
    s1 = re.sub(r'[^\w\s]+|[\d]+', r'', str_row).lower().strip()
    word_arr = s1.split()
    words = [morph.parse(w)[0].normal_form for w in word_arr if w not in russian_stopwords]
    return " ".join(words)

def predict_text_api(text):
    p = preprocessing_text(text)
    X = vectorizer.transform([p]).toarray()
    probs = model.predict(X, verbose=0)[0]
    best_idx = np.argmax(probs)
    label = label_encoder.inverse_transform([best_idx])[0]
    classes = label_encoder.inverse_transform(np.arange(len(probs)))
    prob_dict = {str(classes[i]): float(probs[i]) for i in range(len(probs))}
    return {"label": str(label), "probabilities": prob_dict}

import gradio as gr

def gradio_interface(text):
    res = predict_text_api(text)
    label = res["label"]
    probs = res["probabilities"]
    out = f"✅ Predicted label: {label}\n\n📊 Probabilities:\n"
    sorted_probs = dict(sorted(probs.items(), key=lambda x: x[1], reverse=True))
    for k, v in sorted_probs.items():
        bar = "█" * int(v*20)
        out += f"{k}: {v:.4f} {bar}\n"
    return out

In [17]:
iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(lines=5, placeholder="Введите предложение..."),
    outputs="text",
    title="Text_analizator V1.5, built 14.0.2, Release_2025",
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.43.0, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://505fc66d1d21d29e3a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
